In [3]:
import pickle

In [4]:
import csv
import csv
smiles = []
with open('./data/token/atom_In_Smiles_1M_tokken','rb') as file:
  tok_smiles = pickle.load(file)

In [5]:
import atomInSmiles
smiles = train[0]

# SMILES -> atom-in-SMILES 
ais_tokens = atomInSmiles.encode(smiles) # '[NH2;!R;C] [CH2;!R;CN] [C;!R;COO] ( = [O;!R;C] ) [OH;!R;C]'

# atom-in-SMILES -> SMILES
decoded_smiles = atomInSmiles.decode(ais_tokens) #'NCC(=O)O'

assert smiles == decoded_smiles

NameError: name 'train' is not defined

In [ ]:
smiles = smiles[1:]

In [6]:
from tqdm import tqdm
tok_smile = []
for i in tqdm(smiles):
    tok_smile.append(atomInSmiles.encode(i).split(' '))

0it [00:00, ?it/s]


In [ ]:
## Processing the token
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

2023-11-15 02:43:29.092037: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 02:43:29.127269: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 02:43:29.127295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 02:43:29.127321: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 02:43:29.133745: I tensorflow/core/platform/cpu_feature_g

In [14]:
from tdc.single_pred import Tox
train,tox_info = Tox(name = 'herg_karim' ).get_data(format = 'DeepPurpose')
train_set = rk.smile_tokenize(train)

Found local copy...
Loading...
Done!


NameError: name 'rk' is not defined

In [16]:
tox_info.dtype

dtype('int64')

In [9]:
train_set = []
for i in train:
    tok_smile.append(atomInSmiles.encode(i).split(' '))

In [1]:

## Drop out len 1
drop_train = []
for index,sentence in enumerate(tok_smiles):
    if len(sentence)<=1:
        drop_train.append(index)
        print(index)
drop_sentence = []  

temp_ = []
for i,index in enumerate(drop_train):
    temp_.append(drop_train[index-i])
    tok_smile.pop(index-i)


tokenized_doc = tok_smiles
print('총 샘플 수 :',len(tokenized_doc))

NameError: name 'tok_smiles' is not defined

In [ ]:
tokenized_doc[0]

for i in tokenized_doc:
    i.insert(0,'<CAD>')

In [2]:
for i in tokenized_doc:
    i.append('<END>')

NameError: name 'tokenized_doc' is not defined

In [12]:
temp_dict = {}
index = 0
for i in tokenized_doc[:10000000]:
    for j in i:
        try:
            temp_dict[j]
        except:
            temp_dict[j] = index
            index+=1

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(tok_smiles)


word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
# index encoding
encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [17]:
vocab_size = len(word2idx) + 1 
print('Word size :', vocab_size)

Word size : 167


In [18]:
## Save the result

with open('./Large_mol_tokken/AtomInsmiles/encoded.pkl','wb') as file:
    pickle.dump(encoded,file)
    
with open('./Large_mol_tokken/AtomInsmiles/words.pkl','wb') as file:
    pickle.dump(word2idx,file)



In [19]:
## load the result
import pickle

with open('./Large_mol_tokken/AtomInsmiles/encoded.pkl','rb') as file:
    encoded = pickle.load(file)
    
with open('./Large_mol_tokken/AtomInsmiles/words.pkl','rb') as file:
    word2idx = pickle.load(file)

In [20]:
## model

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
embedding_dim = 256

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size+1000, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size+1000, embedding_dim)(c_inputs)




from IPython.display import SVG
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 256)       298752      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 256)       298752      ['input_2[0][0]']                
                                                                                              

2023-11-16 14:17:48.673340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 14:17:49.589547: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 308 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:ca:00.0, compute capability: 8.0


In [24]:
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import skipgrams
import numpy as np
#validation
skip_temp = []
first_em_val = []
second_em_val = []
y_123_val = []


for itter in range(1):
    skip_gram = []
    first_em = []
    second_em = []
    y_123 = []
    
    
    for i in tqdm(range(len(encoded))):
        skip_gram.append(skipgrams(encoded[i], vocabulary_size=vocab_size, window_size=3))
        
        
    for _, elem in tqdm(enumerate(skip_gram)):
        if len(elem[1]) == 0:
            print('pass')
            continue
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        try:
            labels = np.array(elem[1].get(),dtype='int32')
        except:
            labels = np.array(elem[1],dtype='int32')
        for i in first_elem:
            first_em.append(i)
        for j in second_elem:
            second_em.append(j)
        for k in labels:
            y_123.append(k)
            
    second_em =np.array(second_em,dtype='int32')
    first_em =np.array(first_em,dtype='int32')
    y_123 = np.array(y_123,dtype='int32')


    with tf.device("/device:GPU:0"):
        model.fit([first_em,second_em],y_123,epochs=40,batch_size=10000)
    model.save('Large_mol_tokken/AtomInsmiles/L_model')

100%|██████████| 1470284/1470284 [11:57<00:00, 2048.78it/s]
1470284it [02:45, 8866.07it/s]


Epoch 1/40
37475/37475 [==============================] - 447s 12ms/step - loss: 0.2520 - acc: 0.8998
Epoch 2/40
37475/37475 [==============================] - 321s 9ms/step - loss: 0.2508 - acc: 0.9001
Epoch 3/40
37475/37475 [==============================] - 322s 9ms/step - loss: 0.2507 - acc: 0.9002
Epoch 4/40
37475/37475 [==============================] - 319s 9ms/step - loss: 0.2507 - acc: 0.9002
Epoch 5/40
37475/37475 [==============================] - 299s 8ms/step - loss: 0.2507 - acc: 0.9002
Epoch 6/40
37475/37475 [==============================] - 346s 9ms/step - loss: 0.2507 - acc: 0.9002
Epoch 7/40
37475/37475 [==============================] - 290s 8ms/step - loss: 0.2506 - acc: 0.9002
Epoch 8/40
37475/37475 [==============================] - 325s 9ms/step - loss: 0.2506 - acc: 0.9002
Epoch 9/40
37475/37475 [==============================] - 326s 9ms/step - loss: 0.2506 - acc: 0.9002
Epoch 10/40
37475/37475 [==============================] - 305s 8ms/step - loss: 0.2506 - 

In [ ]:
skip_gram = []
first_em = []
second_em = []
y_123 = []
for i in tqdm(range(len(encoded_add))):
    skip_gram.append(skipgrams(encoded_add[i], vocabulary_size=vocab_size, window_size=3))
        
for _, elem in tqdm(enumerate(skip_gram)):
    if len(elem[1]) == 0:
        print('pass')
        continue
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    try:
        labels = np.array(elem[1].get(),dtype='int32')
    except:
        labels = np.array(elem[1],dtype='int32')
    for i in first_elem:
        first_em.append(i)
    for j in second_elem:
        second_em.append(j)
    for k in labels:
        y_123.append(k)
        
second_em =np.array(second_em,dtype='int32')
first_em =np.array(first_em,dtype='int32')
y_123 = np.array(y_123,dtype='int32')


with tf.device("/device:GPU:0"):
    model.fit([first_em,second_em],y_123,epochs=40,batch_size=1000000,validation_data=([first_em_val,second_em_val], y_123_val))
model.save('Large_mol_tokken/L_model_tuned')

In [ ]:
# model save
import gensim
from tensorflow.keras import models
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle


f = open('./Large_mol_tokken/AtomInsmiles/w2v.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, 256))
vectors = model.get_weights()[0]
for word, i in word2idx.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

# 모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./Large_mol_tokken/AtomInsmiles/w2v.txt', binary=False)